In [4]:
import numpy as np
import pandas as pd

In [103]:
def calculate_performance_loss(baseline_acc, current_acc):
    """Calculate normalized performance loss."""
    #print('check: baseline_acc: ', baseline_acc, 'current_acc: ', current_acc)
    return (current_acc - baseline_acc) / baseline_acc if baseline_acc != 0 else 0

def calculate_differential_losses(pruning_results, baseline):
    """
    Calculate the differential of performance losses across pruning ratios.
    Arguments:
    pruning_results -- list of tuples, where each tuple contains (pruning_ratio, obj_acc, rel_acc, triplet_acc)
    baseline -- tuple of (baseline_obj_acc, baseline_rel_acc, baseline_triplet_acc)
    """
    # Sorting by pruning ratios for accurate differential calculation
    sorted_results = sorted(pruning_results, key=lambda x: x[0])
    ratios = [x[0] for x in sorted_results]
    losses = [[calculate_performance_loss(baseline[i], x[i + 1]) for i in range(3)] for x in sorted_results]
    
    # Calculating differentials of losses
    diffs = -np.diff(np.array(losses), axis=0)
    diffs = np.vstack([diffs, diffs[-1]])  # Duplicate last differential as an approximation for the last point
    return ratios, losses, diffs

def calculate_model_score(losses, diffs, weights=(1, 1, 5), mu=5, pruning_ratio=0, detailed=False):
    """Calculate scores incorporating the absolute values of losses and their differentials, adjusted by lambda."""
    adjusted_losses = losses
    adjusted_diffs = diffs
    #print(f'pruning ratio: {pruning_ratio}, adjusted losses: {adjusted_losses}, adjusted diffs: {adjusted_diffs}')
    weighted_diff_loss = sum(weights[i] * (adjusted_diffs[i] + adjusted_losses[i]) for i in range(3))
    score = weighted_diff_loss + mu * pruning_ratio
    
    if detailed:
        return score, weighted_diff_loss
    return score

In [104]:

base_dir = '/home/song/Desktop/song/lightweight_3DSSG/visualization/pruning_infer_data/'
# CSV 파일 경로
file_path1 = base_dir + 'SGFN_param60_Structured_Pruning_Results.csv'
file_path2 = base_dir +'Attn+SGFN_param60_Structured_Pruning_Results.csv'
file_path3 = base_dir +'SGPN_param40_Structured_Pruning_results.csv'

# CSV 파일 로드
df1 = pd.read_csv(file_path1).sort_values(by='Pruning Ratio')
df2 = pd.read_csv(file_path2).sort_values(by='Pruning Ratio')
df3 = pd.read_csv(file_path3).sort_values(by='Pruning Ratio')

# 필요한 데이터를 추출하여 리스트로 변환
sgfn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df1.iterrows()
]
sgfn_baseline = (
    df1['3d obj Acc@1'].iloc[0],
    df1['3d rel Acc@1'].iloc[0],
    df1['3d triplet Acc@50'].iloc[0]
)
attn_sgfn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df2.iterrows()
]
attn_sgfn_baseline = (
    df2['3d obj Acc@1'].iloc[0],
    df2['3d rel Acc@1'].iloc[0],
    df2['3d triplet Acc@50'].iloc[0]
)

sgpn_pruning_results = [
    (row['Pruning Ratio'], row['3d obj Acc@1'], row['3d rel Acc@1'], row['3d triplet Acc@50'])
    for index, row in df2.iterrows()
]
sgpn_baseline = (
    df2['3d obj Acc@1'].iloc[0],
    df2['3d rel Acc@1'].iloc[0],
    df2['3d triplet Acc@50'].iloc[0]
)
print("SGFN Pruning Results:", sgfn_pruning_results)
print("SGFN Baseline:", sgfn_baseline)
print("Attention SGFN Pruning Results:", attn_sgfn_pruning_results)
print("Attention SGFN Baseline:", attn_sgfn_baseline)
print("SGPN Pruning Results:", sgpn_pruning_results)
print("SGPN Baseline:", sgpn_baseline)


SGFN Pruning Results: [(0.0, 52.39, 92.14, 89.19), (0.05, 52.39, 92.14, 89.19), (0.1, 52.39, 92.07, 89.18), (0.15, 52.39, 91.79, 89.09), (0.2, 52.39, 90.99, 88.87), (0.25, 52.39, 90.39, 88.66), (0.3, 52.46, 89.63, 88.36), (0.35, 52.46, 88.57, 88.13), (0.4, 52.46, 89.17, 87.92), (0.45, 52.27, 85.12, 87.1), (0.5, 52.27, 83.41, 86.65), (0.55, 52.41, 85.73, 86.51), (0.6, 48.6, 67.92, 84.27), (0.65, 41.85, 67.92, 81.56), (0.7, 32.5, 67.92, 76.68), (0.75, 28.47, 67.92, 74.99)]
SGFN Baseline: (52.39, 92.14, 89.19)
Attention SGFN Pruning Results: [(0.0, 54.6259220231823, 90.18144861434732, 89.50473451985523), (0.05, 54.478398314014754, 88.98170641019286, 89.40062465916415), (0.1, 54.373024236037935, 85.12220514600169, 89.20727777502354), (0.15, 54.3940990516333, 81.01234445491052, 89.00897327846909), (0.2, 54.58377239199157, 73.1247831044569, 88.68672847156809), (0.25, 54.58377239199157, 70.74760795201031, 88.5256060681176), (0.3, 54.64699683877766, 74.58727876654604, 88.30994992811462), (0.35

In [105]:
models_data = {
    "SGPN": {
        "baseline": sgpn_baseline,
        "pruning_results": sgpn_pruning_results
    },
    "SGFN": {
        "baseline": sgfn_baseline,
        "pruning_results": sgfn_pruning_results
    },
    "Attn + SGFN": {
        "baseline": attn_sgfn_baseline,
        "pruning_results": attn_sgfn_pruning_results
    },
    # "VLSAT": {
    #     "baseline": vlsat_baseline,
    #     "pruning_results": vlsat_pruning_results
    # }
    
}

In [110]:
def display_scores(models_data, weights=(0, 0, 3), mu=0.1):
    results = {}
    for model_name, data in models_data.items():
        baseline = data["baseline"]
        pruning_results = data["pruning_results"]
        if not pruning_results or len(pruning_results) < 2:
            print(f"Not enough data to calculate scores for {model_name}.")
            continue
        
        ratios, losses, diffs = calculate_differential_losses(pruning_results, baseline)
        
        # Store detailed score components for each pruning ratio
        detailed_scores = []
        scores = []
        for idx in range(len(ratios)):
            score, components = calculate_model_score(losses[idx], diffs[idx], weights, mu, ratios[idx], detailed=True)
            scores.append(score)
            detailed_scores.append({
                "ratio": ratios[idx],
                "weighted_diff_loss": components,
                "weighted_ratio": mu * ratios[idx],
                "total_score": score
            })
        
        max_score_index = np.argmax(scores)
        best_ratio = ratios[max_score_index]
        best_score = scores[max_score_index]

        results[model_name] = {
            "best_ratio": best_ratio,
            "best_score": best_score,
            "all_scores": scores,
            "detailed_scores": detailed_scores
        }

        print(f"Scores for {model_name}:")
        for entry in detailed_scores:
            print(f"  Pruning Ratio {entry['ratio']}, Weighted Diff-Loss: {entry['weighted_diff_loss']}, "
                  f"Weighted Ratio: {entry['weighted_ratio']:.3f}, Total Score: {entry['total_score']:.3f}")
        print(f"Best Pruning Ratio: {best_ratio}, Highest Score: {best_score:.3f}\n")

    return results

In [111]:
final_results = display_scores(models_data)
#final_results

Scores for SGFN:
  Pruning Ratio 0.0, Weighted Diff-Loss: 0.0, Weighted Ratio: 0.000, Total Score: 0.000
  Pruning Ratio 0.05, Weighted Diff-Loss: 0.0003363605785398892, Weighted Ratio: 0.005, Total Score: 0.005
  Pruning Ratio 0.1, Weighted Diff-Loss: 0.002690884628321981, Weighted Ratio: 0.010, Total Score: 0.013
  Pruning Ratio 0.15, Weighted Diff-Loss: 0.004036326942482495, Weighted Ratio: 0.015, Total Score: 0.019
  Pruning Ratio 0.2, Weighted Diff-Loss: -0.0036999663639416484, Weighted Ratio: 0.020, Total Score: 0.016
  Pruning Ratio 0.25, Weighted Diff-Loss: -0.007736293306424622, Weighted Ratio: 0.025, Total Score: 0.017
  Pruning Ratio 0.3, Weighted Diff-Loss: -0.020181634712411516, Weighted Ratio: 0.030, Total Score: 0.010
  Pruning Ratio 0.35, Weighted Diff-Loss: -0.028590649175916874, Weighted Ratio: 0.035, Total Score: 0.006
  Pruning Ratio 0.4, Weighted Diff-Loss: -0.015136226034308402, Weighted Ratio: 0.040, Total Score: 0.025
  Pruning Ratio 0.45, Weighted Diff-Loss: -0